In [0]:
!pip install pystardog

In [0]:
import stardog
import pandas as pd
import json

In [0]:
with open("Credentials.json") as f:   # change file location
    d = json.load(f)
endpoint = d['endpoint']
username = d['username']
password = d['password']

In [0]:
# Connection settings for read-only user
conn_details = {
    'endpoint': endpoint, 
    'username': username,         
    'password': password          
}

# Create the connection manually
conn = stardog.Connection('Sales', **conn_details)

In [0]:
def human_parse_results(results):
    data=[]
    head=results['head']['vars']
    print("Total records : ",len(results['results']["bindings"]))
    # print(results['results']["bindings"])
    for i in results['results']["bindings"]:
        d1=[]
        for j in head:
            # print(i[j]['value'].split('#')[1])
            d1.append(i[j]['value'])#.split('#')[1])
            if(len(i[j]['value'].split('#'))==1):
                print(i[j]['value'])
        data.append(d1)
    # print(data)
    print(pd.DataFrame(data,columns=head))
# human_parse_results(results)

def get_results(conn,query):
    try:
        print("Query : \n",query)
        results = conn.select(query,
                      reasoning=True, 
                      named_graphs=['https://a.in/sales/data']  
                      )
        print("Output : \n",human_parse_results(results))
        # return results
        #human_parse_results(results)
    except Exception as e:
        print(e)

In [0]:
q_test="""PREFIX ex: <https://a.in/sales/ontology#>
            PREFIX data: <https://a.in/sales/data#>

            SELECT DISTINCT ?productFamilyName ?skuName 
            from <https://a.in/sales/data>
            WHERE 
            {  ?pf a ex:ProductFamily ;      
                ex:productFamilyName ?productFamilyName ;      
                ex:hasProductClass ?pc .  
                ?pc ex:hasProductLine ?pl .  
                ?pl ex:hasProductType ?pt .  
                ?pt ex:hasBrand ?b .  
                ?b ex:hasSKU ?sku .  
                ?sku ex:skuName ?skuName .
                }

                limit 10
            """
get_results(conn,q_test)

In [0]:
# get schema
ont="https://a.in/sales/ontology"
q1="""SELECT ?s ?p ?o
from <{g}>
WHERE {{
?s ?p ?o
}}
""".format(g=ont)
get_results(conn,q1)

In [0]:
graph = "https://a.in/sales/data"
query = """SELECT ?s ?p ?o
FROM <{g}>
WHERE {{
?s ?p ?o
}}
LIMIT 40
""".format(g=graph)
get_results(conn,q1)

In [0]:
conn.close()